<h1 style="font-family:Times New Roman;">Website data parsing</h1>

*Nikolay Sitala 2023, nsitala@ukr.net* 
>Data source https://agriline.ua/

In [ ]:
%pip install -q ipywidgets requests beautifulsoup4

<h2 style="font-family:Times New Roman">Скрипт</h2>

In [ ]:
import csv
import json
import requests
from bs4 import BeautifulSoup, Tag
# import pprint
# from datetime import datetime

prfields = []

resc = []
resc_pull = []
outgoing_data = []


def get_sales_subcategory(id1, tm1):
    try:
        if tm1 is None:
            payload = {'cat_id': id1, 'scurr': 'EUR'}
        else:
            payload = {'cat_id': id1, 'tm': tm1, 'scurr': 'EUR'}

        ret = requests.post(f'https://agriline.ua/sales/', data=payload)
        if ret.status_code == 200:
            return ret.json()
        else:
            return {}
    except Exception as ex:
        print(ex)


def filters_data(url):
    global prfields

    def get_fields(jt, level_show, level_cls, group, parent,
                   fnclass, tag_show, tag_hidden,
                   class_find_show="opt with-found-stat",
                   class_find_hidden="opt hidden with-found-stat"):

        global prfields

        try:
            s = jt.find('div', class_='show-all')
            show_ = s.text
            # print(b.attrs, show_.strip(), f"Level: {level_show}", f"Group: {group}",
            #       'Status: show', parent, 0, 0, sep="|")
            prfields.append([b.attrs, show_.strip(), f"Level: {level_show}",
                                 f"Group: {group}", 'show', parent, 0, 0])

            for v in jt.find('div', class_=fnclass):
                if type(v) == Tag:
                    for k in jt.findAll(tag_show, class_=class_find_show):
                        cmp = k.text
                        # print(v.attrs, cmp.strip(), f"Level: {level_cls}", f"Group: {group}",
                        #       'Status: show', parent, 0, 0, sep="|")
                        prfields.append([v.attrs, cmp.strip(), f"Level: {level_cls}",
                                             f"Group: {group}", 'show', parent, 0, 0])

                    for k in jt.findAll(tag_hidden, class_=class_find_hidden):
                        cmp = k.text
                        # print(v.attrs, cmp.strip(), f"Level: {level_cls}", f"Group: {group}",
                        #       'Status: hidden', parent, 0, 0, sep="|")
                        prfields.append([v.attrs, cmp.strip(), f"Level: {level_cls}", f"Group: {group}",
                                             'hidden', parent, 0, 0])
        except Exception as ex:
            print('Error', ex, fnclass, sep="|")

    def get_fields_extension(form_id, cint, jt, level_show, level_cls,
                             group, parent, fnclass, tag_show, tag_hidden):

        global prfields

        sub_parent1 = None
        sub_name_parent1 = None
        html_with_subopts = None

        ret = jt.find('div', class_=fnclass)
        s = ret.find('div', class_='opts')

        for x in s.children:
            if hasattr(x, "class"):
                tractor = x.text
                opt_class = x.get('class')

                if not 'with-subopts' in opt_class and 'with-found-stat' in opt_class:
                    sub_parent1 = x['data-value']
                    # print('with-found-stat with-found-stat', tractor.strip(), f"Level: {level_cls}",
                    #       f"Group: {group}", 'Status: hidden', parent, 0, 0, sep="|")
                    prfields.append(['with-found-stat with-found-stat', tractor.strip(), f"Level: {level_cls}",
                                         f"Group: {group}", 'hidden', parent, 0, 0])

                if 'with-found-stat' in opt_class and 'with-subopts' in opt_class:
                    sub_parent1 = x['data-value']
                    sub_name_parent1 = tractor.strip()
                    # print('with-found-stat', sub_name_parent1, f"Level: {level_cls}", f"Group: {group}",
                    #       'Status: hidden', parent, 0, 0, sep="|")
                    prfields.append(['with-found-stat', sub_name_parent1, f"Level: {level_cls}",
                                         f"Group: {group}", 'hidden', parent, 0, 0])

                    ret_json = get_sales_subcategory(cint, sub_parent1)
                    if ret_json:
                        html_with_subopts = ret_json['filtersForm'][f'{form_id}']['html']

                if f'subopts-for-{sub_parent1}' in opt_class:
                    for sub_for in x.children:
                        if type(sub_for) == Tag:
                            if sub_for.get('data-value'):
                                sub_name_parent2 = sub_for.get('data-value')
                                sub_for2 = sub_for.text.strip()
                                # print(f'subopts-for-{sub_parent1}', sub_for2, f"Level: {level_cls + 1}",
                                #       f"Group: {group}", 'Status: hidden', sub_name_parent1, 0, 0, sep="|")
                                prfields.append([f'subopts-for-{sub_parent1}', sub_for2,
                                                     f"Level: {level_cls + 1}", f"Group: {group}",
                                                     'hidden', sub_name_parent1, 0, 0])

                                if html_with_subopts:
                                    soup_subopts = BeautifulSoup(html_with_subopts, 'html.parser')
                                    h = soup_subopts.find('div', class_=f'subopts-for-{sub_name_parent2}')
                                    if h:
                                        for meta in h.children:
                                            if meta.name == 'span':
                                                # meta_tractor = meta.text
                                                meta2txt = meta.text.strip()
                                                # print(f'subopts-for-{sub_name_parent2}',
                                                #       meta2txt,
                                                #       f"Level: {level_cls + 2}",
                                                #       f"Group: {group}",
                                                #       'Status: hidden',
                                                #       sub_for2.strip(), 0, 0, sep="|")

                                                prfields.append([f'subopts-for-{sub_name_parent2}',
                                                                 meta2txt,
                                                                     f"Level: {level_cls + 2}",
                                                                     f"Group: {group}",
                                                                     'hidden',
                                                                 sub_for2.strip(), 0, 0])

                                                if meta.get('data-value'):
                                                    sub_name_parent3 = meta.get('data-value')
                                                    meta2 = h.find('div', class_=f'subopts-for-{sub_name_parent3}')
                                                    if meta2:
                                                        for meta3 in meta2:
                                                            if type(meta3) == Tag:
                                                                mt = meta3.text
                                                                meta3txt = mt.strip()
                                                                # print(f'subopts-for-{sub_name_parent3}',
                                                                #       meta3txt,
                                                                #       f"Level: {level_cls + 3}",
                                                                #       f"Group: {group}",
                                                                #       'Status: hidden',
                                                                #       meta2txt, 0, 0, sep="|")

                                                                prfields.append([f'subopts-for-{sub_name_parent3}',
                                                                                 meta3txt,
                                                                                     f"Level: {level_cls + 3}",
                                                                                     f"Group: {group}",
                                                                                     'hidden',
                                                                                 meta2txt, 0, 0])

    # url = 'https://agriline.ua/-/traktori--c201'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    wrapper = soup.find('form', class_='ads-filters-form sales-search with-found-stat')

    header_wrapper = soup.find('h1')
    h1 = header_wrapper.text
    h1s = h1.strip()
    print(url, h1s, sep="|")
    prfields.append([url, h1s])
    # [url, h1s, f"Level: 0", f"Group: 0", 'show', None, 0, 0]

    gr = 0
    cint = 0
    ret_all_json = None

    try:
        c1 = url.split("--c")
        cint = int(c1[1])
        ret_all_json = get_sales_subcategory(int(c1[1]), None)

        # html_with_subopts = ret_json['filtersForm']['trademark']['html']
    except Exception as ex:
        print('Error', ex, sep='|')

    try:
        for b in wrapper.div.children:
            tag = b.find('div')
            if not isinstance(tag, int):
                gr += 1
                m = b.find('div', class_='label')
                new_category = m.span.text
                parent_one = new_category.strip()
                # print(b.attrs, new_category.strip(), f"Level: 1", f"Group: {gr}",
                #       'Status: show', None, 0, 0, sep="|")
                prfields.append([b.attrs, new_category.strip(), f"Level: 1", f"Group: {gr}",
                                     'show', None, 0, 0])

                opts_ = b.attrs['data-field-id']

                if opts_ == 'cat_id':
                    f = "field field-select field-id-cat_id"
                    get_fields_extension(opts_, cint, b, 2, 2, gr, parent_one, f,
                                         'a', 'span')

                elif opts_ == 'trademark':
                    f = "field field-select field-id-trademark"
                    if h1s == 'Послуги':
                        get_fields(b, 2, 2, gr, parent_one, f, 'span', 'span')
                    else:
                        get_fields_extension(opts_, cint, b, 2, 2, gr, parent_one, f,
                                             'a', 'span')

                elif opts_ == 'geo':
                    geo_opts = b.find('div', class_='opts')
                    for x in geo_opts.children:
                        if hasattr(x, "class"):
                            data = x.has_attr('data-value')
                            if data:
                                if x['data-value'] == 'ua':
                                    gr += 1
                                    # Украина
                                    # print(x.attrs, x.text, f"Level: 1", f"Group: {gr}",
                                    #       'Status: show', parent_one, 0, 0, sep="|")
                                    prfields.append([x.attrs, x.text, f"Level: 1", f"Group: {gr}",
                                                         'show', parent_one, 0, 0])

                                    qty = geo_opts.find('div', class_='subopts subopts-for-ua opened')
                                    for v in qty:
                                        if type(v) == Tag:
                                            city_string = v.text
                                            # print(v.attrs, city_string.strip(), f"Level: 2", f"Group: {gr}",
                                            #       'Status: hidden', parent_one, 0, 0, sep="|")
                                            prfields.append(
                                                [v.attrs, city_string.strip(), f"Level: 2", f"Group: {gr}",
                                                 'hidden', parent_one, 0, 0])

                                # if x['data-value'] in ['cgrp1', 'cn', 'cgrp8']:
                                #     gr += 1
                                #     print(x.attrs, x.text, f"Level: 1", f"Group: {gr}",
                                #           'Status: show', parent_one, 0, 0, sep="|")
                                    prfields.append([x.attrs, x.text, f"Level: 1", f"Group: {gr}",
                                                         'show', parent_one, 0, 0])

                elif opts_ == 'adsType':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-adsType", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-adsType", 'a', 'a')
                elif opts_ == 'state':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-state", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-state", 'a', 'a')
                elif opts_ == 'euro':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-euro", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-euro", 'a', 'a')
                elif opts_ == 'pto_speed':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-pto_speed", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-pto_speed", 'a', 'a')
                elif opts_ == 'tire_construction':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-tire_construction", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-tire_construction", 'a', 'a')
                elif opts_ == 'tire_size':
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-tire_size", 'span', 'span')
                    get_fields(b, 2, 2, gr, parent_one,
                               "field field-select field-id-tire_size", 'a', 'a')
                else:
                    slider = ret_all_json['filtersForm'][opts_]
                    if b.find('div', class_='range-inputs'):
                        slider_min = slider['min']
                        slider_max = slider['max']

                        # print(f'field-id-{opts_}', parent_one, f"Level: 1", f"Group: {gr}", 'Status: show',
                        #       parent_one, slider_min, slider_max, sep="|")
                        prfields.append([f'field-id-{opts_}', parent_one,
                                             f"Level: 1", f"Group: {gr}", 'Status: show',
                                         parent_one, slider_min, slider_max])

    except Exception as ex:
        print('Error', ex, sep="|")

    try:
        main_fields = soup.findAll('div', class_='fields-group')
        for b in main_fields:
            gr += 1
            m = b.find('div', class_='title')
            fields_group_title = m.text.strip()
            parent_one = fields_group_title
            # print(b.attrs, fields_group_title, f"Level: 1", f"Group: {gr}",
            #       'Status: show', None, 0, 0, sep="|")
            prfields.append([b.attrs, fields_group_title, f"Level: 1", f"Group: {gr}",
                                 'show', None, 0, 0])

            m1 = b.find('div', class_='fields')
            for fd in m1:
                if hasattr(fd, "class"):
                    data = fd.has_attr('data-field-id')
                    if data:
                        fid = fd.attrs['data-field-id']
                        lab = fd.find('div', class_='label')
                        labs = lab.text.strip()
                        parent_two = labs

                        if 'field-wrapper-range' in fd['class']:
                            slider_min = None
                            slider_max = None

                            if fid:
                                slider = ret_all_json['filtersForm'][fid]
                                slider_min = slider['min']
                                slider_max = slider['max']

                            # print(fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}", 'Status: show',
                            #       parent_one, slider_min, slider_max, sep="|")
                            prfields.append([fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}", 'show',
                                             parent_one, slider_min, slider_max])

                        elif 'field-wrapper-select' in fd['class']:
                            # print(fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}", 'Status: show',
                            #       parent_one, 0, 0, sep="|")
                            prfields.append([fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}", 'show',
                                             parent_one, 0, 0])

                            ret = fd.find('div', {'data-field-id': fd['data-field-id']})

                            mark = 'show'
                            pcls = ["opt with-found-stat", "opt hidden with-found-stat"]
                            for z in pcls:
                                sp = ret.findAll('span', class_=z)
                                for k in sp:
                                    comp = k.text.strip()
                                    if 'hidden' in z:
                                        mark = 'hidden'
                                    # print(k.attrs, comp, f"Level: 3", f"Group: {gr}", f'Status: {mark}',
                                    #       parent_two, 0, 0, sep="|")
                                    prfields.append([k.attrs, comp, f"Level: 3", f"Group: {gr}",
                                                     mark, parent_two, 0, 0])

                        elif 'field-wrapper-checkbox' in fd['class']:
                            # print(fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}", 'Status: show',
                            #       parent_one, 0, 0, sep="|")
                            prfields.append([fd.attrs, labs.strip(), f"Level: 2", f"Group: {gr}",
                                                 'show', parent_one, 0, 0])

    except Exception as ex:
        print(ex)


def create_level(scale, chunk, data):
    """
        Создание уровня
    """
    # chunk = None
    g = {}

    for row in data:
        if len(row) > 2:
            article = row[1]
            level = row[2]

            min = int(row[6])
            max = int(row[7])

            if scale == level:
                parent = row[5]
                if scale == 'Level: 1':
                    parent = chunk

                if g.get(parent):
                    x = g[parent]
                    if max > 0:
                        x[article] = {'min': min, 'max': max}
                    else:
                        x[article] = None
                    g[parent] = x
                else:
                    if not parent == article:
                        if max > 0:
                            g[parent] = {article: {'min': min, 'max': max}}
                        else:
                            g[parent] = {article: None}
                    else:
                        if max > 0:
                            g[parent] = {'min': min, 'max': max}
                        else:
                            g[parent] = {}
    return g


def create_nested_level(obj):
    """ Сбор вложенных уровней снизу вверх """

    high4 = obj['Level: 4']
    low5 = obj['Level: 5']

    for h in low5:
        for k in high4:
            if h in high4[k]:
                p = high4[k]
                p[h] = low5[h]
                high4[k] = p

    high3 = obj['Level: 3']
    low4 = obj['Level: 4']

    for h in low4:
        for k in high3:
            if h in high3[k]:
                p = high3[k]
                p[h] = low4[h]
                high3[k] = p

    high2 = obj['Level: 2']
    low3 = obj['Level: 3']

    for h in low3:
        for k in high2:
            if h in high2[k]:
                p = high2[k]
                p[h] = low3[h]
                high2[k] = p

    high1 = obj['Level: 1']
    low2 = obj['Level: 2']

    for h in low2:
        for k in high1:
            if h in high1[k]:
                p = high1[k]
                p[h] = low2[h]
                high1[k] = p

    # pprint.pprint(high1)
    return high1


def data_agrregation():
    """  Агрегация данных по полю родитель
    """

    data = []
    with open('agriline_top.csv', newline='', encoding='utf-8') as csvfile:
        spam_reader = csv.reader(csvfile, dialect='excel')
        for i, row in enumerate(spam_reader):
            if i > 0:
                data.append(row)

    struct = []

    start_row = 0
    finish_row = 0

    chunk = None
    while finish_row < len(data):
        row = data[finish_row]
        if 'https' in row[0] and len(row) == 2:
            if chunk:

                scales = {f'Level: {b}': None for b in range(5, 0, -1)}

                for d in scales:
                    x = create_level(d, chunk, data[start_row + 1: finish_row])
                    scales[d] = x

                struct.append(create_nested_level(scales))

                # chunk = None
                start_row = finish_row
            chunk = row[1]
        finish_row += 1

    json_object = json.dumps(struct, indent=4)
    with open("reduction_filters.json", "w") as outfile:
        outfile.write(json_object)

    return 0


def start_gather_data():
    # Импорт ссылок из текстового файла
    lines = []
    prfields.append(['Ссылка', 'Раздел', 'Уровень', 'Группа', 'Видимость',
                         'Родитель по полю раздел', 'min', 'max'])

    # now = datetime.now()
    # dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    # print(f"Старт импорта ссылок из agriline_list_url.txt: ", dt_string)

    with open('agriline_list_url.txt', 'r', encoding='utf-8') as txt_file:
        for line in txt_file:
            v = line.rstrip()
            if v:
                lines.append(v)

    # Парсинг фильтров по ссылкам
    for b in lines:
        # now = datetime.now()
        # dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        # print(f"Начало парсинга: ", dt_string)
        filters_data(b)
        # break

    # Запись выгруженных и обработанных данных в файл CSV
    # now = datetime.now()
    # dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    # print(f"Запись данных в файл agriline_top.csv: ", dt_string)

    with open('agriline_top.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for row in prfields:
            writer.writerow(row)
            # break


def rev(data, k):
    global resc
    global resc_pull
    global outgoing_data

    k += 1

    for tech in data:
        # if 'Лісозаготівельна техніка' == tech:
        #     print()

        if tech == 'max' or tech == 'min':
            resc[6] = data['min']
            resc[7] = data['max']

            # if int(data['max']) == 790:
            #     print()

            if tech == 'max':
                resc_pull.append(resc.copy())
            resc[6] = None
            resc[7] = None
        else:

            resc[k] = tech
            if type(data[tech]) is dict:
                # print(k, resc, sep="|")
                resc_pull.append(resc.copy())

                rev(data[tech], k)  # Recursion

                new_pull = resc_pull.copy()
                new_pull.reverse()
                p = 0
                for i, r in enumerate(new_pull):
                    t = len([b for b in r if b])
                    if i >= 0:
                        if p > t:
                            v1 = new_pull[i]
                            v0 = new_pull[i-1]
                            if v1[t-1] == v0[t-1] and v1[p-1] is None:
                                pass
                            else:
                                outgoing_data.append(r)
                                # print(r)  # print(k, t, r)
                        else:
                            outgoing_data.append(r)
                            # print(r)  # print(k, t, r)
                    p = t
                    # print(k, t, r)
                resc_pull = []

            else:
                # print(k, resc, sep="|")
                resc_pull.append(resc.copy())

    resc[k] = None
    return 0


def reassembly_flat_table():
    # Конвертор JSON в плоскую таблицу для Excel, формат CSV

    global resc
    global resc_pull
    global outgoing_data

    f = open("reduction_filters.json", "r")
    data = json.loads(f.read())
    f.close()

    for b in data:
        resc = [None for b in range(1, 9)]
        resc_pull = []

        rev(b, -1)
        # break

    with open('agriline_patch_excel.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for row in outgoing_data:
            writer.writerow(row)


if __name__ == '__main__':
    start_gather_data()  # Парсинг фильтров Agriline
    data_agrregation()  # Создание вложенных объектов для сохранения в JSON
    reassembly_flat_table()  # Пересборка JSON в плоскую таблицу для Excel

In [ ]:
%pip install -q pandas
import pandas as pd

dframe = pd.read_csv("agriline_patch_excel.csv", header=None)
dframe.head()

<h2 style="font-family:Times New Roman">Скачать CSV</h2>

In [ ]:
import pandas
import base64
from IPython.display import FileLink
from ipywidgets import HTML

def create_download_link(df, title="Скачать матрицу в CSV file ", filename="agriline_excel.csv"):
    csv = df.to_csv(sep=';', encoding='utf-8-sig', index=False)
    b64 = base64.b64encode(csv.encode('utf-8-sig'))
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload, title=title, filename=filename)
    return HTML(html)


create_download_link(dframe)